In [1]:
import xml.etree.ElementTree as ET
import csv
import pandas as pd
import traceback
import sys
import numpy as np


In [2]:
#common variables

CUTTOFF=100000000
LOGAMOUNT=1000000

OUTPUT1="output/output_1.csv"
OUTPUT2="output/output_2.csv"
OUTPUT3="output/output_3.csv"
OUTPUT4="output/output_4.csv"
PROCESSEDFINAL="output/processed_body.csv"
EXISTING_TAGS="output/existing_tags.csv"
TAGSFILE = 'datadumps/Tags.xml'
POSTSFILE = 'datadumps/Posts.xml'

In [6]:
tags_context = ET.iterparse(TAGSFILE, events=("start", "end"),
                       parser=ET.XMLParser(encoding='utf-8'))

In [4]:
# lets look at the tags
_, root = next(tags_context)
for index, item in enumerate(tags_context):
    event, elem = item
    if event == "end" and elem.tag == "row":
        print(elem.attrib)
        print('\n')
        
        if index >1 and index!= 0:
            break
    
    

{'Id': '1', 'TagName': '.net', 'Count': '310793', 'ExcerptPostId': '3624959', 'WikiPostId': '3607476'}


{'Id': '2', 'TagName': 'html', 'Count': '1088688', 'ExcerptPostId': '3673183', 'WikiPostId': '3673182'}




In [5]:
with open(EXISTING_TAGS, 'w', newline='', encoding='utf-8') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['TAG'])
    _, root = next(tags_context)
    for index, item in enumerate(tags_context):
        event, elem = item
        if event == "end" and elem.tag == "row":
            tag = elem.attrib.get('TagName', 'None')
            csvwriter.writerow([tag])

In [7]:
search_for_tags_df =pd.read_csv(EXISTING_TAGS)
search_for_tags_df = search_for_tags_df.sort_values('TAG')
search_for_tags_df

,TAG
16566,.a
15157,.app
16357,.aspxauth
11751,.bash-profile
15329,.class-file
...,...
36586,zynq
55385,zynq-ultrascale+
29917,zypper
352,NaN


In [3]:
context = ET.iterparse(POSTSFILE, events=("start", "end"),
                       parser=ET.XMLParser(encoding='utf-8'))

In [4]:
# lets look at the data
_, root = next(context)
for index, item in enumerate(context):
    event, elem = item
    if event == "end" and elem.tag == "row":
        print(elem.attrib)
        print('\n')
        tags = elem.attrib.get('Tags', 'None')
        #remove all '<', '>'
        reworked_tags = tags[1:len(tags)-1].split('><')
        print('tags = ', reworked_tags)
        print('\n')
        
    if index >1 and index!= 0:
        break
    
    


{'Id': '4', 'PostTypeId': '1', 'AcceptedAnswerId': '7', 'CreationDate': '2008-07-31T21:42:52.667', 'Score': '735', 'ViewCount': '59537', 'Body': '<p>I want to use a <code>Track-Bar</code> to change a <code>Form</code>\'s opacity.</p>\n<p>This is my code:</p>\n<pre class="lang-cs prettyprint-override"><code>decimal trans = trackBar1.Value / 5000;\nthis.Opacity = trans;\n</code></pre>\n<p>When I build the application, it gives the following error:</p>\n<blockquote>\n<pre class="lang-none prettyprint-override"><code>Cannot implicitly convert type decimal to double\n</code></pre>\n</blockquote>\n<p>I have tried using <code>trans</code> and <code>double</code>, but then the <code>Control</code> doesn\'t work. This code worked fine in a past VB.NET project.</p>\n', 'OwnerUserId': '8', 'LastEditorUserId': '3072350', 'LastEditorDisplayName': 'Rich B', 'LastEditDate': '2021-02-26T03:31:15.027', 'LastActivityDate': '2021-02-26T03:31:15.027', 'Title': 'How to convert a Decimal to a Double in C#?'

In [8]:
def simpleTagCheck(arr):
    return 'is valid' if len(arr) != len(search_for_tags_df.loc[search_for_tags_df['TAG'].
                                                  isin(arr)]) else 'not valid'




In [10]:
search_for_tags_df.loc[search_for_tags_df['TAG'] == 'data-collection']

,TAG
9594,data-collection


In [4]:

#Platforms & tools &libraries
TAGS = ['google-app-maker', 'appian', 'outsystems', 'zoho', 'mendix', 
        'powerapps', 'quickbase', 'vinyl', 'salesforce-lightning', 'github-copilot','Knack','Airtable']

# Research Related
TAGS += ['machine-learning', 'low-code']

# Keywords
TAGS += ['build-automation', 'sdlc', 'automation','integration','classification',
        'workflow', 'interface','resource-management','user-interface', 'deployment',
         'architecture','security','maintenance', 'data-collection']



COLS = ["Id", "PostTypeId", "AcceptedAnswerId", "ParentId", "CreationDate",
        "DeletionDate", "Score", "ViewCount", "Body", "LastEditDate", "LastActivityDate",
        "Title", "Tags", "AnswerCount", "CommentCount", "FavoriteCount", "ClosedDate",
        "CommunityOwnedDate", "ContentLicense"]

In [9]:
simpleTagCheck(TAGS)

'is valid'

In [ ]:
# Extract data by tags
print("Start Extraction")
postcount =0
with open(OUTPUT1, 'w', newline='', encoding='utf-8') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(COLS)
    _, root = next(context)
    for index, item in enumerate(context):
        event, elem = item
        # prevent duplicates
        if event == "end" and elem.tag == "row":
            tags = elem.attrib.get('Tags', 'None')
            #remove all '<', '>'
            reworked_tags = tags[1:len(tags)-1].split('><')
            for tag in TAGS:
                if tag in reworked_tags:
                    postcount+=1
                    data = []
                    for col in COLS:
                        data.append(elem.attrib.get(col, ''))
                    csvwriter.writerow(data)
                    continue
            elem.clear()
            root.clear()          
        # log progress every 100000 
        if index % LOGAMOUNT == 0 and index != 0:
            print('prossesed', index, 'total_posts_gahtered', postcount)

        if index > CUTTOFF:
            break
                
            
            
print("Extraction Over")

Start Extraction
prossesed 1000000 total_posts_gahtered 4187
prossesed 2000000 total_posts_gahtered 7847
prossesed 3000000 total_posts_gahtered 11233
prossesed 4000000 total_posts_gahtered 14583
prossesed 5000000 total_posts_gahtered 17925
prossesed 6000000 total_posts_gahtered 20922
prossesed 7000000 total_posts_gahtered 23771
prossesed 8000000 total_posts_gahtered 26514
prossesed 9000000 total_posts_gahtered 29219
prossesed 10000000 total_posts_gahtered 31846
prossesed 11000000 total_posts_gahtered 34422
prossesed 12000000 total_posts_gahtered 36753
prossesed 13000000 total_posts_gahtered 39147
prossesed 14000000 total_posts_gahtered 41484
prossesed 15000000 total_posts_gahtered 43801
prossesed 16000000 total_posts_gahtered 46177
prossesed 17000000 total_posts_gahtered 48549
prossesed 18000000 total_posts_gahtered 50804
prossesed 19000000 total_posts_gahtered 53055
prossesed 20000000 total_posts_gahtered 55361
prossesed 21000000 total_posts_gahtered 57520
prossesed 22000000 total_pos

In [6]:
# clean data
colsSet = set(TAGS)
df = pd.read_csv(OUTPUT1)
df

,Id,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,DeletionDate,Score,ViewCount,Body,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate,ContentLicense
0,42,1,77.0,NaN,2008-08-01T12:50:18.587,NaN,291,38845,<p>I am starting a new web application in PHP ...,2015-12-14T20:05:51.823,2021-05-29T06:04:39.960,Best way to allow plugins for a PHP application,<php><plugins><architecture><hook>,8,0,114.0,NaN,NaN,CC BY-SA 3.0
1,328,1,7488.0,NaN,2008-08-02T02:41:34.493,NaN,125,82768,<p>What are some guidelines for maintaining re...,2012-05-04T09:39:26.510,2012-07-18T17:21:23.430,PHP Session Security,<security><php>,13,0,186.0,NaN,NaN,CC BY-SA 3.0
2,549,1,477578.0,NaN,2008-08-02T19:51:50.250,NaN,5508,634447,<h2>Form-based authentication for websites</h2...,2018-04-09T07:41:21.840,2021-07-24T04:22:43.753,The definitive guide to form-based website aut...,<security><http><authentication><language-agno...,11,16,6701.0,2016-08-11T07:19:48.077,2008-10-01T00:39:14.900,CC BY-SA 3.0
3,580,1,586.0,NaN,2008-08-02T23:30:59.090,NaN,30,7803,<p>I wonder how you guys manage deployment of ...,2014-11-25T20:58:47.550,2014-11-25T20:58:47.550,Deploying SQL Server Databases from Test to Live,<sql-server><sql-server-2005><deployment><rele...,14,0,5.0,NaN,NaN,CC BY-SA 2.5
4,2056,1,101561.0,NaN,2008-08-05T10:06:33.020,NaN,2241,525972,"<p>When looking beyond the <a href=""https://en...",2020-08-29T01:23:49.180,2021-05-19T14:23:15.523,What are MVP and MVC and what is the difference?,<user-interface><model-view-controller><design...,24,5,1324.0,NaN,NaN,CC BY-SA 3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88415,22372449,1,NaN,NaN,2014-03-13T08:00:58.233,NaN,3,330,"<p>I am following this <a href=""http://contoso...",2014-08-13T21:18:44.860,2015-11-19T15:52:02.283,Multiple Database Context,<c#><asp.net-mvc-4><architecture><unity-contai...,1,0,0.0,NaN,NaN,CC BY-SA 3.0
88416,22372530,1,NaN,NaN,2014-03-13T08:05:25.760,NaN,0,162,<p>does UI Automation supports Windows Phone 8...,2014-06-02T21:27:21.670,2014-06-02T21:27:21.670,UI Automation + Windows Phone 8?,<user-interface><windows-phone-8><automation>,1,1,NaN,NaN,NaN,CC BY-SA 3.0
88417,22372530,1,NaN,NaN,2014-03-13T08:05:25.760,NaN,0,162,<p>does UI Automation supports Windows Phone 8...,2014-06-02T21:27:21.670,2014-06-02T21:27:21.670,UI Automation + Windows Phone 8?,<user-interface><windows-phone-8><automation>,1,1,NaN,NaN,NaN,CC BY-SA 3.0
88418,22373555,1,24058997.0,NaN,2014-03-13T08:58:06.167,NaN,0,596,<p>Is openSUSE shim bootloader signed with ope...,NaN,2014-06-05T11:27:39.257,openSUSE shim certificate for uefi secure boot,<security><boot><opensuse>,1,0,NaN,NaN,NaN,CC BY-SA 3.0


In [7]:
print('REMOVE QUESTIONS WITH WRONG TAGS')
print('Before', len(df))
indicesToRemove = []
for i in range(len(df)):
    tags = df.iloc[i]['Tags'][1:-1].replace('><', ' ').split()
    f = True
    for tag in tags:
        if tag in colsSet:
            f = False
            break
    if f:
        indicesToRemove.append(i)
df = df.drop(index=indicesToRemove)

print('After', len(df))

REMOVE QUESTIONS WITH WRONG TAGS
Before 88420
After 88420


In [8]:
print('\nREMOVING DUPLICATE SCORES')
print('Before', len(df))
df.drop_duplicates('Id',inplace=True)
print('After', len(df))




REMOVING DUPLICATE SCORES
Before 88420
After 87037


In [9]:
print('\nREMOVING QUESTIONS WITH NEGATIVE SCORES')
print('Before', len(df))
df = df[df['Score'] >= 0]
df.to_csv(OUTPUT2, index=False)
print('After', len(df))


REMOVING QUESTIONS WITH NEGATIVE SCORES
Before 87037
After 84705


In [5]:
df = pd.read_csv(OUTPUT2)
df

,Id,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,DeletionDate,Score,ViewCount,Body,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate,ContentLicense
0,42,1,77.0,NaN,2008-08-01T12:50:18.587,NaN,291,38845,<p>I am starting a new web application in PHP ...,2015-12-14T20:05:51.823,2021-05-29T06:04:39.960,Best way to allow plugins for a PHP application,<php><plugins><architecture><hook>,8,0,114.0,NaN,NaN,CC BY-SA 3.0
1,328,1,7488.0,NaN,2008-08-02T02:41:34.493,NaN,125,82768,<p>What are some guidelines for maintaining re...,2012-05-04T09:39:26.510,2012-07-18T17:21:23.430,PHP Session Security,<security><php>,13,0,186.0,NaN,NaN,CC BY-SA 3.0
2,549,1,477578.0,NaN,2008-08-02T19:51:50.250,NaN,5508,634447,<h2>Form-based authentication for websites</h2...,2018-04-09T07:41:21.840,2021-07-24T04:22:43.753,The definitive guide to form-based website aut...,<security><http><authentication><language-agno...,11,16,6701.0,2016-08-11T07:19:48.077,2008-10-01T00:39:14.900,CC BY-SA 3.0
3,580,1,586.0,NaN,2008-08-02T23:30:59.090,NaN,30,7803,<p>I wonder how you guys manage deployment of ...,2014-11-25T20:58:47.550,2014-11-25T20:58:47.550,Deploying SQL Server Databases from Test to Live,<sql-server><sql-server-2005><deployment><rele...,14,0,5.0,NaN,NaN,CC BY-SA 2.5
4,2056,1,101561.0,NaN,2008-08-05T10:06:33.020,NaN,2241,525972,"<p>When looking beyond the <a href=""https://en...",2020-08-29T01:23:49.180,2021-05-19T14:23:15.523,What are MVP and MVC and what is the difference?,<user-interface><model-view-controller><design...,24,5,1324.0,NaN,NaN,CC BY-SA 3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84700,22372436,1,22374569.0,NaN,2014-03-13T08:00:13.823,NaN,0,126,<p>I need some architectural advice on how to ...,2020-06-20T09:12:55.060,2014-03-14T19:18:16.367,Transfer data between websites,<c#><.net><api><architecture>,1,0,NaN,NaN,NaN,CC BY-SA 3.0
84701,22372449,1,NaN,NaN,2014-03-13T08:00:58.233,NaN,3,330,"<p>I am following this <a href=""http://contoso...",2014-08-13T21:18:44.860,2015-11-19T15:52:02.283,Multiple Database Context,<c#><asp.net-mvc-4><architecture><unity-contai...,1,0,0.0,NaN,NaN,CC BY-SA 3.0
84702,22372530,1,NaN,NaN,2014-03-13T08:05:25.760,NaN,0,162,<p>does UI Automation supports Windows Phone 8...,2014-06-02T21:27:21.670,2014-06-02T21:27:21.670,UI Automation + Windows Phone 8?,<user-interface><windows-phone-8><automation>,1,1,NaN,NaN,NaN,CC BY-SA 3.0
84703,22373555,1,24058997.0,NaN,2014-03-13T08:58:06.167,NaN,0,596,<p>Is openSUSE shim bootloader signed with ope...,NaN,2014-06-05T11:27:39.257,openSUSE shim certificate for uefi secure boot,<security><boot><opensuse>,1,0,NaN,NaN,NaN,CC BY-SA 3.0


In [6]:
answerIdDict = {}
for index, row in df.iterrows():
    # access data using column names
    temp = row['AcceptedAnswerId']
    # collect only not nan vlues with indexes
    if not (temp != temp):
        answerIdDict[int(temp)] = index

print('removed nan, reworked total:', len(answerIdDict))



removed nan, reworked total: 54965


In [9]:
def updater(header, data, filename):
    with open(filename, "w", newline="", encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=header)
        writer.writeheader()
        writer.writerows(data)
        
print("Start Processing")
postcount =0 
with open(OUTPUT2, newline='', encoding='utf-8') as file:
    readData = [row for row in csv.DictReader(file)]
    _, root = next(context)
    for index, item in enumerate(context):
        event, elem = item
        
        if event == "end" and elem.tag == "row":
            ansId = int(elem.attrib.get('Id', 'None'))
            if ansId in answerIdDict:
                postcount += 1
                
                ansindex = answerIdDict[ansId]
                readData[ansindex]['AcceptedAnswer'] = elem.attrib.get("Body", '')
                readData[ansindex]['AcceptedAnswerCreationDate'] = elem.attrib.get('CreationDate', '')
                readHeader = readData[ansindex].keys()
                updater(readHeader, readData, OUTPUT3)
            elem.clear()
            root.clear()
            if index % LOGAMOUNT == 0 and index != 0:
                print('prossesed:', index, 'total_answers_gahtered:', postcount)
                
            if index > CUTTOFF:
                break
            
print("End Processing")

Start Processing


KeyboardInterrupt: 

In [10]:
df = pd.read_csv(OUTPUT3)
df

,Id,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,DeletionDate,Score,ViewCount,Body,LastEditDate,...,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate,ContentLicense,AcceptedAnswer,AcceptedAnswerCreationDate
0,42,1,77.0,NaN,2008-08-01T12:50:18.587,NaN,291,38845,<p>I am starting a new web application in PHP ...,2015-12-14T20:05:51.823,...,Best way to allow plugins for a PHP application,<php><plugins><architecture><hook>,8,0,114.0,NaN,NaN,CC BY-SA 3.0,NaN,NaN
1,328,1,7488.0,NaN,2008-08-02T02:41:34.493,NaN,125,82768,<p>What are some guidelines for maintaining re...,2012-05-04T09:39:26.510,...,PHP Session Security,<security><php>,13,0,186.0,NaN,NaN,CC BY-SA 3.0,NaN,NaN
2,549,1,477578.0,NaN,2008-08-02T19:51:50.250,NaN,5508,634447,<h2>Form-based authentication for websites</h2...,2018-04-09T07:41:21.840,...,The definitive guide to form-based website aut...,<security><http><authentication><language-agno...,11,16,6701.0,2016-08-11T07:19:48.077,2008-10-01T00:39:14.900,CC BY-SA 3.0,NaN,NaN
3,580,1,586.0,NaN,2008-08-02T23:30:59.090,NaN,30,7803,<p>I wonder how you guys manage deployment of ...,2014-11-25T20:58:47.550,...,Deploying SQL Server Databases from Test to Live,<sql-server><sql-server-2005><deployment><rele...,14,0,5.0,NaN,NaN,CC BY-SA 2.5,NaN,NaN
4,2056,1,101561.0,NaN,2008-08-05T10:06:33.020,NaN,2241,525972,"<p>When looking beyond the <a href=""https://en...",2020-08-29T01:23:49.180,...,What are MVP and MVC and what is the difference?,<user-interface><model-view-controller><design...,24,5,1324.0,NaN,NaN,CC BY-SA 3.0,<h2>Model-View-Presenter</h2>\n<p>In <strong>M...,2008-09-19T12:46:52.687
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14973,2662629,1,2662807.0,NaN,2010-04-18T14:55:39.047,NaN,1,574,<p>Ok this is something weird. I got authlogic...,2010-04-18T15:20:25.280,...,"Authlogic, logout, credential capture and secu...",<ruby-on-rails><security><openid><authlogic-oid>,2,1,2.0,NaN,NaN,CC BY-SA 2.5,NaN,NaN
14974,2663017,1,2707735.0,NaN,2010-04-18T16:39:01.637,NaN,3,231,"<p>We have a requirement, wherein the administ...",2010-04-22T04:24:51.900,...,log in as a proxy for a certain user,<security><authentication><jakarta-ee><seam>,3,1,1.0,NaN,NaN,CC BY-SA 2.5,NaN,NaN
14975,2663677,1,2663688.0,NaN,2010-04-18T19:46:11.420,NaN,6,6226,<p>I'm writing a TemplateEngine that will allo...,2018-06-19T11:11:29.633,...,Changing property type in class that implement...,<c#><interface>,3,3,NaN,NaN,NaN,CC BY-SA 4.0,NaN,NaN
14976,2664099,1,2664270.0,NaN,2010-04-18T21:47:30.843,NaN,25,28602,<p>I've found numerous posts on stackoverflow ...,2020-11-04T04:00:48.047,...,How can we store password other than plain text?,<linux><security><authentication>,8,3,4.0,NaN,NaN,CC BY-SA 4.0,NaN,NaN


In [11]:
questionIdDict = {}
for index, row in df.iterrows():
    temp = row['Id']
    if not (temp != temp):
        questionIdDict[int(temp)] = index
        
print('removed nan, reworked total:',len(questionIdDict))

removed nan, reworked total: 14978


In [12]:
questionIdDict

{42: 0,
 328: 1,
 549: 2,
 580: 3,
 2056: 4,
 2222: 5,
 2447: 6,
 2525: 7,
 2540: 8,
 2688: 9,
 2933: 10,
 2970: 11,
 3027: 12,
 3362: 13,
 3486: 14,
 3607: 15,
 3667: 16,
 3725: 17,
 3781: 18,
 3867: 19,
 4046: 20,
 4785: 21,
 4850: 22,
 4913: 23,
 5013: 24,
 5078: 25,
 5242: 26,
 5313: 27,
 5329: 28,
 5611: 29,
 5855: 30,
 5880: 31,
 6085: 32,
 6729: 33,
 6847: 34,
 7174: 35,
 7440: 36,
 7489: 37,
 7551: 38,
 7596: 39,
 8004: 40,
 8066: 41,
 8214: 42,
 8318: 43,
 8451: 44,
 8472: 45,
 8968: 46,
 9240: 47,
 9376: 48,
 9433: 49,
 9543: 50,
 9846: 51,
 10059: 52,
 10443: 53,
 10634: 54,
 10731: 55,
 10901: 56,
 11152: 57,
 11263: 58,
 11330: 59,
 11462: 60,
 11689: 61,
 11801: 62,
 12008: 63,
 12009: 64,
 12243: 65,
 12428: 66,
 12523: 67,
 12815: 68,
 13347: 69,
 13607: 70,
 13893: 71,
 14350: 72,
 14617: 73,
 14656: 74,
 14828: 75,
 14893: 76,
 15053: 77,
 15056: 78,
 15139: 79,
 15390: 80,
 15690: 81,
 15815: 82,
 16003: 83,
 16145: 84,
 16155: 85,
 16199: 86,
 16556: 87,
 16965: 88,

In [13]:
print("Start Processing")
postcount =0 
with open(OUTPUT4, 'w', newline='', encoding='utf-8') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(COLS)
    _, root = next(context)
    for index, item in enumerate(context):
        event, elem = item
        if event == "end" and elem.tag == "row":
            parentId = int(elem.attrib.get('ParentId', '0'))
            
            if parentId in questionIdDict:
                postcount += 1
                data = []
                for col in COLS:
                    data.append(elem.attrib.get(col, ''))
                csvwriter.writerow(data)
                continue
            
        
            elem.clear()
            root.clear()
        # progress
        if index % LOGAMOUNT == 0 and index != 0:
            print('prossesed:', index, 'total_questions_gahtered:', postcount)
                
        if index > CUTTOFF:
            break
print("End Processing")
 

Start Processing
prossesed: 1000000 total_questions_gahtered: 13632
prossesed: 2000000 total_questions_gahtered: 24065
prossesed: 3000000 total_questions_gahtered: 33299
prossesed: 4000000 total_questions_gahtered: 40479
prossesed: 5000000 total_questions_gahtered: 41050
prossesed: 6000000 total_questions_gahtered: 41437
prossesed: 7000000 total_questions_gahtered: 41798
prossesed: 8000000 total_questions_gahtered: 42082
prossesed: 9000000 total_questions_gahtered: 42330
prossesed: 10000000 total_questions_gahtered: 42535


KeyboardInterrupt: 

In [14]:
df = pd.read_csv(OUTPUT3, low_memory=False)

answers_df = df.copy()
answers_df = answers_df[['Id', 'AcceptedAnswer']]
answers_df = answers_df.dropna()
answers_df.columns = ['id', 'raw']
answers_df.insert(1, 'qa', 'a')
answers_df.id = answers_df.id.apply(np.int64)
print(answers_df.head())
print(len(answers_df))

questions_df = df.copy()
questions_df = questions_df[['Id', 'Body', 'Title']]
questions_df['titlePlusQuestion'] = questions_df[['Title', 'Body']].apply(lambda x: ' '.join(x), axis=1)
questions_df = questions_df.drop(['Title', 'Body'], axis=1)
questions_df.columns = ['id', 'raw']
questions_df.insert(1, 'qa', 'q')
print(questions_df.head())
print(len(questions_df))

df = pd.concat([questions_df,answers_df])
print(len(df))
df.to_csv(PROCESSEDFINAL, index=False)

       id qa                                                raw
4    2056  a  <h2>Model-View-Presenter</h2>\n<p>In <strong>M...
16   3667  a  <p>trunk for development, and a branch (produc...
42   8214  a  <p>The following blog post gives quite a few g...
48   9376  a  <p>I use ILMerge for almost all of my differen...
87  16556  a  <p>I used to use it a lot.  This designer work...
1067
     id qa                                                raw
0    42  q  Best way to allow plugins for a PHP applicatio...
1   328  q  PHP Session Security <p>What are some guidelin...
2   549  q  The definitive guide to form-based website aut...
3   580  q  Deploying SQL Server Databases from Test to Li...
4  2056  q  What are MVP and MVC and what is the differenc...
14978
16045
